# groupby 공부
pandas로 dplyr처럼 그룹바이에 익숙해지기 위한 공부

링크 : https://rfriend.tistory.com/390?category=675917

In [6]:
# importing libraries

import numpy as np
import pandas as pd

# making sample dataset

df = pd.DataFrame(data=np.arange(20).reshape(4, 5),
            columns = ['c1', 'c2', 'c3', 'c4', 'c5'], 
            index = ['r1', 'r2', 'r3', 'r4'])


In [7]:
df

,c1,c2,c3,c4,c5
r1,0,1,2,3,4
r2,5,6,7,8,9
r3,10,11,12,13,14
r4,15,16,17,18,19


## (1). Dicts를 이용한 groupby 예제

### 1-1. 행 기준 Dicts를 이용한 groupby 집계 axis=0

In [10]:
mapping_dict_row = {'r1':'row_g1',
                    'r2':'row_g1',
                    'r3':'row_g2',
                    'r4':'row_g2'}

grouped_by_row = df.groupby(mapping_dict_row)
grouped_by_row.sum()

,c1,c2,c3,c4,c5
row_g1,5,7,9,11,13
row_g2,25,27,29,31,33


### 1-2. 열 기준 Dicts를 이용한 groupby 집계 axis=1

In [11]:
mapping_dict_col = {'c1':'col_g1',
                    'c2':'col_g1',
                    'c3':'col_g2',
                    'c4':'col_g2',
                    'c5':'col_g2'}
grouped_by_col = df.groupby(mapping_dict_col, axis=1)
grouped_by_col.sum()

,col_g1,col_g2
r1,1,9
r2,11,24
r3,21,39
r4,31,54


## (2). Series를 이용한 groupby 집계